In [3]:
import requests
import json
import pandas as  pd
import time
# Running flask app from powershell
# $env:FLASK_APP = "api"
# $env:FLASK_ENV = "development"
# flask run

# Running from terminal on OSX
# export FLASK_APP=api
# export FLASK_ENV=development
# flask run

In [4]:
url = 'http://127.0.0.1:5000/createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [5]:
response.json()

{'signingKey_hex': '3e8c1225d8e717a6b2e36165f773a493fb0ac1e64fc169c14a7c594c3d7e3247',
 'traderId': '1',
 'verifyKey_hex': '2487103fe8becbf09a3be3e3c722028b278ca93129481b555cd596b814a730d1'}

In [6]:
url = 'http://127.0.0.1:5000/createMarket'

mkId = 1
# copy from response
sk = response.json()['signingKey_hex']
vk = response.json()['verifyKey_hex']
tId = response.json()['traderId']

In [7]:
content_makemarket = {"signingKey_hex": sk,
                    "traderId": tId, 
                    "verifyKey_hex": vk,
                    "marketRootId": mkId, 
                    "marketBranchId": 1, 
                    "marketMin": 0,
                    "marketMax":1}

response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)

In [8]:
pd.DataFrame(response.json(), index=[0])


,checks,marketBranchId,marketMax,marketMin,marketRootId,traderId
0,ProbablyASignatureError,1,1,0,1,1


In [9]:
# Create a trade
url = 'http://127.0.0.1:5000/createTrade'

content_maketrade = {"signingKey_hex": sk, "traderId": tId, "verifyKey_hex": vk,
  "marketRootId": mkId, "marketBranchId": 1, "price": 0.551, "quantity":1}

In [10]:
# Post trade
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [11]:
# View json response
pd.DataFrame(response.json(), index=[0])

,checks,marketBranchId,marketRootId,price,quantity,traderId
0,ProbablyASignatureError,1,1,0.551,1,1


In [12]:
# Create a matched trade
url = 'http://127.0.0.1:5000/createTrade'
content_maketrade['quantity'] = -1
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [13]:
response.json()

{'checks': 'ProbablyASignatureError',
 'marketBranchId': 1,
 'marketRootId': 1,
 'price': 0.551,
 'quantity': -1,
 'traderId': '1'}

In [14]:
# Create many matched trades
url = 'http://127.0.0.1:5000/createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 0.1286461353302002 seconds.
matched trade: 1 in 0.10862302780151367 seconds.
matched trade: 2 in 0.1051938533782959 seconds.
matched trade: 3 in 0.09927582740783691 seconds.
matched trade: 4 in 0.0927281379699707 seconds.
matched trade: 5 in 0.11298799514770508 seconds.
matched trade: 6 in 0.09368562698364258 seconds.
matched trade: 7 in 0.10071229934692383 seconds.
matched trade: 8 in 0.11627626419067383 seconds.
matched trade: 9 in 0.12430071830749512 seconds.


In [15]:
url = 'http://127.0.0.1:5000/createMarket'

for i in range(4):
    t = time.time()

    content_makemarket = {"signingKey_hex": sk,
      "traderId": int(tId), "verifyKey_hex": vk,
      "marketRootId": i+1, "marketBranchId": 1, "marketMin": 0, "marketMax":0.7}

    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    if response.json()['checks'] == 'True':
        print('market created: ' + str(i+1) + ' in ' + str(time.time() - t) + ' seconds.')
    else:
        print('failed to create market ' +str(i+1))
    

failed to create market 1
failed to create market 2
failed to create market 3
failed to create market 4


In [16]:
response.json()

{'checks': 'ProbablyASignatureError',
 'marketBranchId': 1,
 'marketMax': 0.7,
 'marketMin': 0,
 'marketRootId': 4,
 'traderId': 1}

In [17]:
url = 'http://127.0.0.1:5000/viewMarketBounds'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)



In [18]:
a = pd.read_json(response.json())
a

,marketRootId,marketBranchId,marketMin,marketMax


In [19]:
url = 'http://127.0.0.1:5000/viewOrderBook'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [20]:
a = pd.read_json(response.json())
a

,marketRootId,marketBranchId,price,quantity,traderId


In [177]:
# Create many matched trades
url = 'http://127.0.0.1:5000/createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = 1
    content_maketrade['price'] = 0.44
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 1.2885279655456543 seconds.
matched trade: 1 in 1.328688144683838 seconds.
matched trade: 2 in 1.280871868133545 seconds.
matched trade: 3 in 1.2452399730682373 seconds.
matched trade: 4 in 1.1938281059265137 seconds.
matched trade: 5 in 1.2085411548614502 seconds.
matched trade: 6 in 1.283189058303833 seconds.
matched trade: 7 in 1.2652091979980469 seconds.
matched trade: 8 in 1.3009788990020752 seconds.
matched trade: 9 in 1.3005650043487549 seconds.


In [185]:
url = 'http://127.0.0.1:5000/viewOpenTrades'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [186]:
pd.read_json(response.json())

,marketRootId,marketBranchId,price,quantity,traderId
0,1,1,0.44,10,3


In [182]:
url = 'http://127.0.0.1:5000/viewMatchedTrades'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [183]:
pd.read_json(response.json())

,marketRootId,marketBranchId,price,quantity,traderId
0,1,1,0.551,0,3


In [184]:
tId

'3'